In [1]:
import multipledispatch
import functools
import numpy as np
import operator
import dask
import dask
import dask.array as da

In [2]:
import sys
import os
PROJECT_PATH = os.path.realpath(os.path.join(os.getcwd(), '..'))
sys.path.append(PROJECT_PATH)

In [3]:
import scs_dask.linalg.linear_operator as linop

namespace_atoms = dict()
dispatch = functools.partial(multipledispatch.dispatch, namespace=namespace_atoms)

In [4]:
@dispatch(da.Array, str, str)
def graph_dot(array, input_key, output_key, transpose=False, **options):
    r""" Build dask graph storing as output a linear operator applied to input.

    Args:
        array (:obj:`da.Array`): Matrix
        input_key (:obj:`str`): Key, in some dask graph, of an input
            vector assumed to be compatibly sized and chunked with
            the array.
        output_key (:obj:`str`): Key of an output vector.
        transpose (:obj:`bool`, optional): If ``True``, form output as
            :math:`w = A^Tz`; by default form :math:`y = Ax`.

    Returns:
        :obj:`dask.sharedict.Sharedict`: dask graph of matrix-vector
        product assigned to output vector.
    """
    if transpose:
        idx_out, idx_arr, idx_in = 'j', 'ij', 'i'
        transform = da.transpose
    else:
        idx_out, idx_arr, idx_in = 'i', 'ij', 'j'
        transform = None
    blks_in = (array.numblocks[1 - int(transpose)],)

    dsk_out = da.core.top(
            da.core.dotmany,
            output_key, idx_out,
            array.name, idx_arr,
            input_key, idx_in,
            leftfunc=transform,
            numblocks={array.name: array.numblocks, input_key: blks_in})
    dsk = dask.sharedict.merge(array.dask)
    dsk.update_with_key(dsk_out, output_key)
    return dsk

In [5]:
@dispatch(linop.DLODense, str, str)
def graph_dot(dense_op, input_key, output_key, transpose=False, **options):
    """ Implementation of :func:`graph_dot` for a dense linear operator.
    """
    return graph_dot(dense_op.data, input_key, output_key, transpose=transpose)

In [6]:
@dispatch(linop.DLODiagonal, str, str)
def graph_dot(diag_op, input_key, output_key, **options):
    """ Implementation of :func:`graph_dot` for a diagonal linear operator.
    """
    vec = diag_op.data
    dsk_out = da.core.top(
            operator.mul, output_key, 'i', vec.name, 'i', input_key, 'i',
            numblocks={vec.name: vec.numblocks, input_key: vec.numblocks})
    dsk = dask.sharedict.merge(diag_op.dask)
    dsk.update_with_key(dsk_out, output_key)
    return dsk

In [7]:
@dispatch(linop.DLOGram, str, str)
def graph_dot(gram_op, input_key, output_key, **options):
    """ Implementation of :func:`graph_dot` for a gram operator.
    """
    mid_key = gram_op.name + '-gramA-' + input_key
    dsk_Ax = graph_dot(
            gram_op.data, input_key, mid_key, transpose=gram_op.transpose)
    dsk_AAx = graph_dot(
            gram_op.data, mid_key, output_key, transpose=(not gram_op.transpose))
    return dask.sharedict.merge(dsk_Ax, dsk_AAx)

In [8]:
@dispatch(linop.DLORegularizedGram, str, str)
def graph_dot(gram_op, input_key, output_key, **options):
    """ Implementation of :func:`graph_dot` for a regularized operator.
    """
    mid_key = gram_op.name + '-gramAA-' + input_key
    blocks = (gram_op.numblocks[0],)
    def wrap_gram(data):
        return data if isinstance(data, linop.DLOGram) else linop.DLOGram(data)
    def add_regularization(AAxi, xi):
        return AAxi + gram_op.regularization * xi

    dsk_AAx = graph_dot(wrap_gram(gram_op.data), input_key, mid_key)
    dsk_IAAx = da.core.top(
            add_regularization, output_key, 'i', mid_key, 'i', input_key, 'i',
            numblocks={mid_key: blocks, input_key: blocks})
    dsk = dask.sharedict.merge(dsk_AAx)
    dsk.update_with_key(dsk_IAAx, output_key)
    return dsk

In [9]:
A = da.random.random((100, 50), chunks=10)
B = da.random.random((50, 100), chunks=10)
x = da.random.random(50, chunks=10)
y = da.random.random(100, chunks=10)

In [10]:
dsk_Ax = dask.sharedict.merge(graph_dot(A, x.name, 'Ax'), x.dask)
dsk_Ax = dask.sharedict.merge(dsk_Ax, x.dask)
Ax = da.Array(dsk_Ax, 'Ax', shape=(A.shape[0],), chunks=(A.chunks[0],), dtype=A.dtype)
(diff,) = dask.compute(Ax - A.dot(x))
assert np.linalg.norm(diff) < 1e-15

In [11]:
dsk_ATy = graph_dot(A, y.name, 'ATy', transpose=True)
dsk_ATy = dask.sharedict.merge(dsk_ATy, y.dask)
ATy = da.Array(dsk_ATy, 'ATy', shape=(A.shape[1],), chunks=(A.chunks[1],), dtype=A.dtype)
(diff, ATy) = dask.compute(ATy - da.transpose(A).dot(y), ATy)
assert np.linalg.norm(diff) < 1e-15 * (1 + np.linalg.norm(ATy))

In [12]:
dsk_Ax = graph_dot(linop.DLODense(A), x.name, 'Ax')
dsk_Ax = dask.sharedict.merge(dsk_Ax, x.dask)
Ax = da.Array(dsk_Ax, 'Ax', shape=(A.shape[0],), chunks=(A.chunks[0],), dtype=A.dtype)
(diff,) = dask.compute(Ax - A.dot(x))
assert np.linalg.norm(diff) < 1e-15

In [13]:
DD = da.random.random(x.shape, chunks=x.chunks)
dsk_Dx = graph_dot(linop.DLODiagonal(DD), x.name, 'Dx')
dsk_Dx = dask.sharedict.merge(dsk_Dx, x.dask)
Dx = da.Array(dsk_Dx, 'Dx', shape=x.shape, chunks=x.chunks, dtype=x.dtype)
(diff,) = dask.compute(Dx - DD * x)
assert np.linalg.norm(diff) < 1e-15

In [14]:
# dsk_AAx for A'A
dsk_AAx = graph_dot(linop.DLOGram(A, name='ATA'), x.name, 'AAx')
dsk_AAx = dask.sharedict.merge(dsk_AAx, x.dask)
AAx = da.Array(dsk_AAx, 'AAx', shape=(A.shape[1],), chunks=(A.chunks[1],), dtype=A.dtype)
diff = AAx - da.transpose(A).dot(A.dot(x))
(diff, AAx) = dask.compute(diff, AAx)
assert np.linalg.norm(diff) < 1e-15 * (1 + np.linalg.norm(AAx))

In [15]:
# dsk_BBx for BB'
dsk_BBx = graph_dot(linop.DLOGram(B, name='BBT'), x.name, 'BBx')
dsk_AAx = dask.sharedict.merge(dsk_BBx, x.dask)
BBx = da.Array(dsk_BBx, 'BBx', shape=x.shape, chunks=x.chunks, dtype=x.dtype)
diff = BBx - B.dot(da.transpose(B).dot(x))
(diff, BBx) = dask.compute(diff, BBx)
assert np.linalg.norm(diff) < 1e-15 * (1 + np.linalg.norm(BBx))

In [16]:
mu = 1 + np.random.random()

In [17]:
# dsk_IAAx for mu * I + A'A
muIAA = linop.DLORegularizedGram(A, regularization=mu)
dsk_muIAAx = graph_dot(muIAA, x.name, 'muIAAx')
dsk_muIAAx = dask.sharedict.merge(dsk_muIAAx, x.dask)
muIAAx = da.Array(dsk_muIAAx, 'muIAAx', shape=(A.shape[1],), chunks=(A.chunks[1],), dtype=A.dtype)
diff = muIAAx - (mu * x + da.transpose(A).dot(A.dot(x)))
(diff, muIAAx) = dask.compute(diff, muIAAx)
assert np.linalg.norm(diff) < 1e-15 * (1 + np.linalg.norm(muIAAx))

In [18]:
# dsk_IBBx for mu * I + BB')
muIBB = linop.DLORegularizedGram(B, regularization=mu)
dsk_muIBBx = graph_dot(muIBB, x.name, 'muIBBx')
dsk_muIBBx = dask.sharedict.merge(dsk_muIBBx, x.dask)
muIBBx = da.Array(dsk_muIBBx, 'muIBBx', shape=(B.shape[0],), chunks=(B.chunks[0],), dtype=B.dtype)
diff = muIBBx - (mu * x + B.dot(da.transpose(B).dot(x)))
(diff, muIBBx) = dask.compute(diff, muIBBx)
assert np.linalg.norm(diff) < 1e-15 * (1 + np.linalg.norm(muIBBx))